# VGG model 테스트

In [67]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

- in_channel = 들어가는 이미지의 채널 수
- out_channel = 나오는 이미지의 채널 수 (사실 상 사용하고자하는 필터의 개수 라고 봐도됨)   
https://gaussian37.github.io/dl-pytorch-conv2d/

In [78]:
a = torch.rand(4,4)
a = a.view(2,2,2,2)
a

tensor([[[[0.9754, 0.2203],
          [0.0147, 0.3246]],

         [[0.7968, 0.8747],
          [0.4509, 0.3188]]],


        [[[0.2569, 0.9425],
          [0.4605, 0.6515]],

         [[0.8878, 0.1607],
          [0.1273, 0.4537]]]])

In [84]:

#torch.Size([32, 3, 224, 224]) = img
input = torch.ones(1, 1, 3, 3)
filter = torch.ones(1, 1, 3, 3)
filter



tensor([[[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]]])

In [69]:
input = Variable(input, requires_grad=True)
input

tensor([[[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]]], requires_grad=True)

In [70]:
filter = Variable(filter)

In [71]:
out = F.conv2d(input, filter)
out

tensor([[[[9.]]]], grad_fn=<ThnnConv2DBackward0>)

In [72]:
out.backward()
print(out.grad_fn)

In [73]:
print(input.grad)

tensor([[[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]]])


# VGG + tensorboard + torchinfo + Epoch당 걸리는 시간
11.10(목)
- 코드 리뷰
    - 하나부터 열까지 다 이해함을 목표로 진행.
1. [궁금한 점 (언더바 차이)](#궁금한-점(언더바-차이))
2. [궁금한 점 (init_weight)](#궁금한-점-(init_weight))

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

import torchvision
import torchvision.transforms as tr
import torchvision.models as models


from VGG import VGG11, VGG13, VGG16, VGG19

import os
from skimage import io, transform
from matplotlib import pyplot as plt
import re
import PIL
from PIL import Image
import glob



path = "/home/sldev1/Project/hyeongeun_test/data/FER"


''' random seed fixed'''
import random
import torch.backends.cudnn as cudnn
import numpy as np

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
np.random.seed(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

In [4]:
device = 'cuda'
model = VGG16()
learning_rate =0.001
# VGG11, VGG13, VGG16, VGG19 중에 택일하여 사용

In [5]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform(m.weight)
    if isinstance(m, nn.Conv2d):
        torch.nn.init.kaiming_uniform(m.weight)

# for param in model.parameters():
#     print(param)

model.apply(init_weights)
for param in model.parameters():
    print(param)


# model.parameters()
# print(optimizer)

NameError: name 'init_weights' is not defined

# 궁금한 점(언더바 차이)
1. kaiming_uniform_, kaiming_uniform에서 언더바 있고 없고 차이점 궁금   
2. Vgg에서 Linear 다음도 ReLU인데, 그럼 kaiming_uniform 써야되는거 아닌지??   
    - 가져온 코드라 왜 이렇게 짰는지 궁금해서

In [44]:
# Simple Learning Rate Scheduler
def lr_scheduler(optimizer, epoch):
    lr = learning_rate
    if epoch == 50:
        lr /= 2
    if epoch == 100:
        lr /= 2
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr 

# Xavier, 가중치 초기화 부분
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform(m.weight)
    if isinstance(m, nn.Conv2d):
        torch.nn.init.kaiming_uniform(m.weight)
    # if isinstance(m, nn.ReLU):
    #     torch.nn.init.kaiming_uniform_

In [3]:
def img_load(img_path):

    jpg = glob.glob(img_path+'*.jpg')
    sort_jpg = sorted(jpg, key=lambda s: int(re.findall(r'\d+', s)[1]))
    #print("img_load def is play")
    return sort_jpg


def label_load(label_path):
    #enumerate #파이썬내장함수.. 강민규사원님은 이걸로 쓰셨는데 느리더래
    label = glob.glob(label_path+'*exp.npy')
    sort_label = sorted(label,key=lambda s: int(re.findall(r'\d+', s)[1]))
    #print("label_load def is play")
    return sort_label

class MyFERDataset(Dataset):
    
    def __init__(self, img_path_base, label_path_base, train=None, transform=None):
        super(MyFERDataset, self).__init__()
        
        if train == True:
            self.img_path = img_path_base+'/train_set/images/'
            self.label_path = label_path_base+'/train_set/annotations/'
        else:
            self.img_path = img_path_base+'/val_set/images/'
            self.label_path = label_path_base+'/val_set/annotations/'

        self.img = img_load(self.img_path)
        self.label = label_load(self.label_path)
        self.transform = transform
        
    
    def __getitem__(self, idx):
        img_read = io.imread(self.img[idx]) 
        label_read = np.load(self.label[idx])
        label_read = label_read.astype(np.int64)
        label_tr = torch.from_numpy(label_read)
        
        
        if self.transform:
            img_tr = self.transform(img_read)
        
        
        return img_tr, label_tr
    

    def __len__(self):
        return len(self.label)
    
    if __name__ == "__main__":
        print('hi, i\'m main')

hi, i'm main


In [4]:
transform_train = tr.Compose([tr.ToTensor(),
                              tr.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])
transform_test = tr.Compose([tr.ToTensor()])


train_dataset = MyFERDataset(img_path_base = path,
                              label_path_base = path,
                              train=True,
                              transform=transform_train)
test_dataset = MyFERDataset(img_path_base = path,
                              label_path_base = path,
                              train=False,
                              transform=transform_test)

In [5]:
len(train_dataset)

287651

In [6]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

In [7]:
dataiter = iter(train_loader)
images, labels = dataiter.next()

In [8]:
labels.size()
#print(images)

torch.Size([32])

In [9]:
print(labels)

tensor([0, 1, 0, 1, 1, 1, 0, 3, 0, 0, 6, 0, 1, 1, 1, 1, 0, 1, 0, 2, 3, 6, 1, 1,
        0, 0, 2, 1, 5, 1, 1, 6])


In [10]:
device = 'cuda'
model = VGG16()
# VGG11, VGG13, VGG16, VGG19 중에 택일하여 사용

# 궁금한 점 (init_weight)
1. apply 함수 시퀀스 궁금. (상준선임, 민규사원님께 물어볼 것)   
    - 참고 자료 : [apply docs](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)   
    - 참고 자료 : [apply source docs](https://pytorch.org/docs/stable/_modules/torch/nn/modules/module.html#Module.apply)
2. 얼마만큼 재귀적으로 불러오는지 어떻게 아는지 궁금.
    - apply시 한번만 초기화하는지?
    - 그걸 어떻게 아는지?

In [11]:
model.apply(init_weights) # 재귀적으로 모델 전체 가중치를 초기화
model = model.to(device)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  


In [12]:
learning_rate = 0.001
num_epoch = 200
model_name = 'VGG_FER.pth'

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
train_loss = 0
valid_loss = 0
correct = 0
total_cnt = 0
best_acc = 0

# 여기서 걸리는 시간 측정

In [ ]:
import time
start = time.time() #시작 시간 저장
# Train
writer = SummaryWriter()
for epoch in range(num_epoch):
    
    if epoch == 1:
        start = time.time() #시작 시간 저장
    if epoch == 2:
        epoch_time_for_1 = time.time()-start
        timer = epoch_time_for_1 * num_epoch
        print("학습에 총 걸리는 시간 :", timer)

    
    
    
    print(f"====== { epoch+1} epoch of { num_epoch } ======")
    model.train()
    lr_scheduler(optimizer, epoch)
    train_loss = 0
    valid_loss = 0
    correct = 0
    total_cnt = 0
    
    
    #writer.add_scalar('Loss/train', train_loss, epoch)
    #writer.add_scalar('Loss/valid', np.random.random(), n_iter)
    #writer.add_scalar('Accuracy/train', valid_loss, epoch)
    #writer.add_scalar('Accuracy/valid', np.random.random(), n_iter)
    
    # Train Phase
    for step, batch in enumerate(train_loader):
        #  input and target
        batch[0], batch[1] = batch[0].to(device), batch[1].to(device) # batch[0]은 이미지 / batch[1]은 label
        optimizer.zero_grad()
        
        logits = model(batch[0])
        loss = loss_fn(logits, batch[1])
        loss.backward()
        
        optimizer.step()
        train_loss += loss.item()
        _, predict = logits.max(1)
        
        total_cnt += batch[1].size(0)
        correct +=  predict.eq(batch[1]).sum().item()
        
        
        if step % 1000 == 0 and step != 0:
            print(f"\n====== { step } Step of { len(train_loader) } ======")
            print(f"Train Acc : { correct / total_cnt }")
            print(f"Train Loss : { loss.item() / batch[1].size(0) }")

    
    train_acc_visual = (correct / total_cnt) * 100
    train_loss_visual = loss.item() / batch[1].size(0)
    writer.add_scalar('acc/train', train_acc_visual, epoch)
    writer.add_scalar('loss/train', train_loss_visual, epoch)

        
            
    correct = 0
    total_cnt = 0
    
# Test Phase
    with torch.no_grad():
        model.eval()
        for step, batch in enumerate(test_loader):
            # input and target
            batch[0], batch[1] = batch[0].to(device), batch[1].to(device)
            total_cnt += batch[1].size(0)
            logits = model(batch[0])
            valid_loss += loss_fn(logits, batch[1])
            _, predict = logits.max(1)
            correct += predict.eq(batch[1]).sum().item()
        valid_acc = correct / total_cnt
        print(f"\nValid Acc : { valid_acc }")    
        print(f"Valid Loss : { valid_loss / total_cnt }")
        valid_acc_visual = valid_acc * 100
        valid_loss_visual = valid_loss / total_cnt
        writer.add_scalar('acc/valid', valid_acc_visual, epoch)
        writer.add_scalar('loss/valid', valid_loss_visual, epoch)

        if(valid_acc > best_acc):
            best_acc = valid_acc
            #torch.save(model, model_name)
            torch.save(model.state_dict(), model_name)
            print("Model Saved!")
    


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

====== 1 epoch of 200 ======

====== 1000 Step of 8990 ======
Train Acc : 0.49063436563436563
Train Loss : 0.032309286296367645

====== 2000 Step of 8990 ======
Train Acc : 0.5499437781109445
Train Loss : 0.05138865113258362

====== 3000 Step of 8990 ======
Train Acc : 0.5767556647784072
Train Loss : 0.030065201222896576

====== 4000 Step of 8990 ======
Train Acc : 0.5919145213696576
Train Loss : 0.03900831937789917


In [15]:
!tensorboard --logdir=runs

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

----
## 참고자료
- 논문 구현
    - [링크](https://cryptosalamander.tistory.com/158)
- tensorboard
    - [링크](https://gaussian37.github.io/dl-pytorch-observe/#tensorboard%EB%A5%BC-%ED%86%B5%ED%95%9C-%ED%95%99%EC%8A%B5-%ED%98%84%ED%99%A9-%ED%99%95%EC%9D%B8-1)